In [ ]:
/content/drive/MyDrive/DATASCI/data

In [2]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [3]:
!pip install opencv-contrib-python --upgrade

In [5]:
!pip uninstall opencv-python
!pip uninstall opencv-contrib-python

Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/cv2/*
    /usr/local/lib/python3.11/dist-packages/opencv_python-4.11.0.86.dist-info/*
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libQt5Core-e7f476e2.so.5.15.16
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libQt5Gui-1ce8cbfe.so.5.15.16
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libQt5Test-9ac3ed15.so.5.15.16
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libQt5Widgets-cd430389.so.5.15.16
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libQt5XcbQpa-b18b5a78.so.5.15.16
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libX11-xcb-0e257303.so.1.0.0
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libXau-00ec42fe.so.6.0.0
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libavcodec-76c43bf0.so.59.37.100
    /usr/loca

In [1]:
!pip install opencv-contrib-python --upgrade

In [20]:
# Initialize face detector and recognizer
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [21]:
# Load and preprocess images
def load_images(folder, label):
    images = []
    labels = []
    for filename in os.listdir(f'data/{folder}'):
        img = cv2.imread(f'data/{folder}/{filename}')
        if img is not None:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                face_roi = gray[y:y+h, x:x+w]
                resized = cv2.resize(face_roi, (100, 100))
                images.append(resized)
                labels.append(label)
    return images, labels

In [22]:
# Load all mood data
happy_imgs, happy_labels = load_images('happy', 'happy')
sad_imgs, sad_labels = load_images('sad', 'sad')
angry_imgs, angry_labels = load_images('angry', 'angry')
confused_imgs, confused_labels = load_images('confused', 'confused')
not_you_imgs, not_you_labels = load_images('not_you', 'not_you')

In [23]:
# Prepare data for face recognition (your face vs others)
your_faces = happy_imgs + sad_imgs + angry_imgs + confused_imgs
your_labels = [0]*len(your_faces)
other_faces = not_you_imgs
other_labels = [1]*len(other_faces)

# Train face recognizer
recognizer.train(your_faces, np.array(your_labels))

In [24]:
# Prepare data for mood classification
X = [img.flatten() for img in your_faces]
y = happy_labels + sad_labels + angry_labels + confused_labels

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train mood classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier()

In [25]:
# Evaluate performance
y_pred = clf.predict(X_test)
print("Mood Classification Report:")
print(classification_report(y_test, y_pred))

Mood Classification Report:
              precision    recall  f1-score   support

       angry       1.00      1.00      1.00         2
    confused       1.00      1.00      1.00         1
       happy       1.00      1.00      1.00         1
         sad       1.00      1.00      1.00         4

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



In [26]:
# Complete prediction function
def predict_mood(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return "Error loading image"

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return "No face detected"

    x,y,w,h = faces[0]
    face = gray[y:y+h, x:x+w]
    face = cv2.resize(face, (100, 100))

    # Face verification
    label, confidence = recognizer.predict(face)
    if label == 1 or confidence > 70:
        return "Face not recognized as yours"

    # Mood prediction
    return clf.predict([face.flatten()])[0]

In [27]:
predict_mood('testpic.jpg')

'Face not recognized as yours'